In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
import tensorflow_datasets as tfds

In [13]:
(train_dataset, dev_dataset), ds_info = tfds.load(
    "food101",
    split=["train", "validation"],
    shuffle_files=True,
    as_supervised=True,   # tuple-format (image tensors, label)
    with_info=True       # metadata
)

In [14]:
def preprocess(image, label):
    # Resize images to the size expected by DenseNet
    image = tf.image.resize(image, [224, 224])
    # Normalize pixel values
    image = tf.keras.applications.densenet.preprocess_input(image)
    return image, label

train_dataset = train_dataset.map(preprocess).batch(32)
dev_dataset = dev_dataset.map(preprocess).batch(32)


In [37]:
base_model = DenseNet121(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(ds_info.features['label'].num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False


In [28]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [29]:
history = model.fit(train_dataset, validation_data=dev_dataset, epochs=10)


Epoch 1/10
2368/2368 [==============================] - 99s 40ms/step - loss: 1.9368 - accuracy: 0.5105 - val_loss: 1.4653 - val_accuracy: 0.6043
Epoch 2/10
2368/2368 [==============================] - 93s 39ms/step - loss: 1.4992 - accuracy: 0.6050 - val_loss: 1.3202 - val_accuracy: 0.6400
Epoch 3/10
2368/2368 [==============================] - 93s 39ms/step - loss: 1.3497 - accuracy: 0.6410 - val_loss: 1.3420 - val_accuracy: 0.6349
Epoch 4/10
2368/2368 [==============================] - 93s 39ms/step - loss: 1.2324 - accuracy: 0.6677 - val_loss: 1.3207 - val_accuracy: 0.6467
Epoch 5/10
2368/2368 [==============================] - 93s 39ms/step - loss: 1.1173 - accuracy: 0.6948 - val_loss: 1.3213 - val_accuracy: 0.6518
Epoch 6/10
2368/2368 [==============================] - 93s 39ms/step - loss: 1.0282 - accuracy: 0.7157 - val_loss: 1.4126 - val_accuracy: 0.6413
Epoch 7/10
2368/2368 [==============================] - 93s 39ms/step - loss: 0.9406 - accuracy: 0.7353 - val_loss: 1.4532 -

In [30]:
evaluation = model.evaluate(dev_dataset)
print(f"Test Accuracy: {evaluation[1] * 100:.2f}%")

790/790 [==============================] - 23s 29ms/step - loss: 1.6672 - accuracy: 0.6289
Test Accuracy: 62.89%


In [38]:
N = 2
for layer in base_model.layers[-N:]:  # Unfreeze the last N layers
    layer.trainable = True

In [ ]:
additional_epochs = 15
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_dataset, epochs=additional_epochs, validation_data=dev_dataset)

Epoch 1/15
2368/2368 [==============================] - 100s 40ms/step - loss: 2.0136 - accuracy: 0.4952 - val_loss: 1.3962 - val_accuracy: 0.6258
Epoch 2/15
2368/2368 [==============================] - 93s 39ms/step - loss: 1.5594 - accuracy: 0.5910 - val_loss: 1.3377 - val_accuracy: 0.6397
Epoch 3/15
2368/2368 [==============================] - 93s 39ms/step - loss: 1.4008 - accuracy: 0.6285 - val_loss: 1.3466 - val_accuracy: 0.6376
Epoch 4/15
2368/2368 [==============================] - 92s 39ms/step - loss: 1.2831 - accuracy: 0.6529 - val_loss: 1.2873 - val_accuracy: 0.6584
Epoch 5/15
2368/2368 [==============================] - 92s 39ms/step - loss: 1.1767 - accuracy: 0.6790 - val_loss: 1.3252 - val_accuracy: 0.6491
Epoch 6/15
2368/2368 [==============================] - 93s 39ms/step - loss: 1.0749 - accuracy: 0.7015 - val_loss: 1.3622 - val_accuracy: 0.6499
Epoch 7/15
2368/2368 [==============================] - 92s 39ms/step - loss: 0.9881 - accuracy: 0.7215 - val_loss: 1.4121 